In [1]:
%matplotlib inline
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.applications.resnet50 import *
from tensorflow.keras.datasets import cifar100
# from keras.datasets import cifar10
# from keras.applications.resnet50 import *
# from keras.layers import *
# from keras.models import *
# import keras

import numpy as np
import pandas as pd
keras.__version__

'2.2.4-tf'

In [2]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

In [3]:
x_train.shape, y_train.shape

((50000, 32, 32, 3), (50000, 1))

In [4]:
y_train

array([[19],
       [29],
       [ 0],
       ...,
       [ 3],
       [ 7],
       [73]])

In [30]:
def get_model_bin_classify(feature_extractor):
    x = feature_extractor.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(10, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=feature_extractor.input, outputs=x)
    model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['acc'])
    return model

def get_train_samples(X_train, y_train, cls_li):
    mask = np.zeros(y_train.shape, dtype=np.bool)
    for cls in cls_li:
        mask += (y_train==cls)
    mask = np.squeeze(mask)
    return X_train[mask], y_train[mask]
    
def eva_bin_on_cls(model, X_valid, y_valid, cls_li, pos_samples):
    re = {}
    for cls in cls_li:
        mask = (y_valid==cls)
        mask = np.squeeze(mask)
        X_valid_cur_sess = X_valid[mask]
        y_valid_cur_sess = y_valid[mask]
        y_valid_cur_sess = convert_to_bin_samples(y_valid_cur_sess, pos_samples)
        loss, metric = model.evaluate(x=X_valid_cur_sess, y=y_valid_cur_sess, verbose=0)
        re[cls]= (loss, metric)
        # print('Evaluation on %s th class loss: %f. Acc: %f' % (cls, loss, metric))
    return re

def convert_to_bin_samples(labels, pos_samples):
    # arg1 = original labels, pos_samples = samples mark possitive
    re_label = np.zeros(labels.shape, dtype=np.bool)
    for it in pos_samples:
        re_label += (labels == it)
    return re_label

In [31]:
test_model = get_model_bin_classify(ResNet50(weights='imagenet', include_top=False))

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


# Continue exp on bin classifier with larger datasets
+ Method: Train a bin classifier incrementally with equal number of pos/neg samples every learning session
+ Purpose: observe whether the performance of previous seen classes will converge.

In [6]:
import matplotlib.pyplot as plt
import random

In [32]:
def print_eva_re_dic(dic):
    for cls, (loss, metric) in dic.items():
        print('Evaluation on %s th class loss: %f. Acc: %f' % (cls, loss, metric))

def train_schemer(train_set, eva_set, pos_sample=[0], scheme=None, label_pool=None):
    # scheme is a iterable obj consists with iterable objs reps label samples of each learning sessions
    (x_train, y_train), (x_test, y_test) = train_set, eva_set
    # seen_labels = set()
    if scheme is None:
        pass
    else:
        for neg_sample in scheme:
            X_train_sess, y_train_sess = get_train_samples(x_train, y_train, pos_sample+neg_sample)
            y_train_sess = convert_to_bin_samples(y_train_sess, pos_sample)
            trained_model = yield X_train_sess, y_train_sess, neg_sample
#             print('evaluating model')
#             seen_labels.union(set(neg_sample))
#             if trained_model is None:
#                 continue
#             re_dict = eva_bin_on_cls(trained_model, x_test, y_test, list(seen_labels), pos_sample)
#             print_eva_re_dic(re_dict)
        yield None    

In [13]:
[[x] for x in range(1,100)]

[[1],
 [2],
 [3],
 [4],
 [5],
 [6],
 [7],
 [8],
 [9],
 [10],
 [11],
 [12],
 [13],
 [14],
 [15],
 [16],
 [17],
 [18],
 [19],
 [20],
 [21],
 [22],
 [23],
 [24],
 [25],
 [26],
 [27],
 [28],
 [29],
 [30],
 [31],
 [32],
 [33],
 [34],
 [35],
 [36],
 [37],
 [38],
 [39],
 [40],
 [41],
 [42],
 [43],
 [44],
 [45],
 [46],
 [47],
 [48],
 [49],
 [50],
 [51],
 [52],
 [53],
 [54],
 [55],
 [56],
 [57],
 [58],
 [59],
 [60],
 [61],
 [62],
 [63],
 [64],
 [65],
 [66],
 [67],
 [68],
 [69],
 [70],
 [71],
 [72],
 [73],
 [74],
 [75],
 [76],
 [77],
 [78],
 [79],
 [80],
 [81],
 [82],
 [83],
 [84],
 [85],
 [86],
 [87],
 [88],
 [89],
 [90],
 [91],
 [92],
 [93],
 [94],
 [95],
 [96],
 [97],
 [98],
 [99]]

In [ ]:
pos_sample = [0]
trainer = train_schemer((x_train, y_train), (x_test, y_test), scheme=[[x] for x in range(1, 100)])

seen_labels = set(pos_sample)
while True:
    print("*"*20)
    print('Next session begins:', end=' ')
    sess_inst = next(trainer)
    if sess_inst is None:
        break
    x_train_sess, y_train_sess, neg_sample = sess_inst
    print('newly arrived neg samples: ', neg_sample)
    test_model.fit(x=x_train_sess, y=y_train_sess, epochs=1, verbose=0)
    seen_labels.union(set(neg_sample))
    re_dict = eva_bin_on_cls(test_model, x_test, y_test, list(seen_labels), pos_sample)
    print_eva_re_dic(re_dict)

********************
Next session begins: newly arrived neg samples:  [1]


In [29]:
test_model.evaluate?